In [25]:
import pandas as pd
import hashlib
from supabase import create_client, Client
import os
from dotenv import load_dotenv

load_dotenv()

# Load Supabase credentials from .env
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Load and preprocess data
df = pd.read_csv("preprocessed_redditData-removedtopic.csv")

# Ensure datetime columns
df["created_utc"] = pd.to_datetime(df["created_utc"])
df["year"] = df["created_utc"].dt.year
df["month"] = df["created_utc"].dt.month
df["day"] = df["created_utc"].dt.day

# Remove NaNs (avoid JSON serialization errors)
df = df.dropna(subset=["subreddit", "text", "year", "month", "day"])

# ========== DIMENSION TABLES ==========

# DIM: Subreddit
subreddits = df[["subreddit"]].drop_duplicates().rename(columns={"subreddit": "name"})
supabase.table("dim_subreddit").upsert(subreddits.to_dict(orient="records"), on_conflict="name").execute()

# DIM: Content (with MD5 hash)
df["content_hash"] = df["text"].apply(lambda x: hashlib.md5(x.encode("utf-8")).hexdigest())
contents = df[["text", "content_hash"]].drop_duplicates().rename(columns={"text": "content"})
supabase.table("dim_content").upsert(contents.to_dict(orient="records"), on_conflict="content_hash").execute()

# DIM: Year
years = df[["year"]].drop_duplicates()
supabase.table("dim_year").upsert(years.to_dict(orient="records"), on_conflict="year").execute()

# DIM: Month
months = df[["month"]].drop_duplicates()
supabase.table("dim_month").upsert(months.to_dict(orient="records"), on_conflict="month").execute()

# DIM: Day
days = df[["day"]].drop_duplicates()
supabase.table("dim_day").upsert(days.to_dict(orient="records"), on_conflict="day").execute()

# ========== BUILD FACT TABLE ==========

# Fetch dimension ID mappings
def get_id_map(table, key_col):
    data = supabase.table(table).select("*").execute().data
    return {row[key_col]: row["id"] for row in data}

subreddit_map = get_id_map("dim_subreddit", "name")
content_map = get_id_map("dim_content", "content_hash")
year_map = get_id_map("dim_year", "year")
month_map = get_id_map("dim_month", "month")
day_map = get_id_map("dim_day", "day")

# Map IDs to dataframe
df["subreddit_id"] = df["subreddit"].map(subreddit_map)
df["content_id"] = df["content_hash"].map(content_map)
df["year_id"] = df["year"].map(year_map)
df["month_id"] = df["month"].map(month_map)
df["day_id"] = df["day"].map(day_map)

# Build fact records
fact_cols = ["subreddit_id", "content_id", "year_id", "month_id", "day_id"]
fact_posts = df[fact_cols].dropna().drop_duplicates().astype(int)
supabase.table("fact_post").insert(fact_posts.to_dict(orient="records")).execute()


APIResponse[TypeVar](data=[{'id': 1, 'subreddit_id': 1, 'content_id': 1, 'year_id': 1, 'month_id': 1, 'day_id': 1}, {'id': 2, 'subreddit_id': 1, 'content_id': 2, 'year_id': 1, 'month_id': 2, 'day_id': 2}, {'id': 3, 'subreddit_id': 1, 'content_id': 3, 'year_id': 1, 'month_id': 1, 'day_id': 3}, {'id': 4, 'subreddit_id': 1, 'content_id': 4, 'year_id': 1, 'month_id': 3, 'day_id': 4}, {'id': 5, 'subreddit_id': 1, 'content_id': 5, 'year_id': 1, 'month_id': 4, 'day_id': 5}, {'id': 6, 'subreddit_id': 1, 'content_id': 6, 'year_id': 1, 'month_id': 5, 'day_id': 6}, {'id': 7, 'subreddit_id': 1, 'content_id': 7, 'year_id': 1, 'month_id': 6, 'day_id': 5}, {'id': 8, 'subreddit_id': 1, 'content_id': 8, 'year_id': 1, 'month_id': 4, 'day_id': 6}, {'id': 9, 'subreddit_id': 1, 'content_id': 9, 'year_id': 1, 'month_id': 7, 'day_id': 7}, {'id': 10, 'subreddit_id': 1, 'content_id': 10, 'year_id': 1, 'month_id': 7, 'day_id': 8}, {'id': 11, 'subreddit_id': 1, 'content_id': 11, 'year_id': 1, 'month_id': 2, 'day